Import Libraries

In [5]:
import cv2
import pickle
import numpy as np
import cvzone

Parking Space Picker

In [6]:
img=cv2.imread(r"C:\Users\umite\Desktop\vs_yapay_zeka\parking_space_counter\carParkImg.png") # Getting our park area image.

width,height=107,48 # Width and height of only for one car parking area.

try:
    with open("CarParkPos","rb") as f: # Getting position information, if we have a position list .
            pos_list=pickle.load(f)
except:
    pos_list=[] # New position list, if we don't have position list.

def mouse_click(events,x,y,flags,params):
    if events==cv2.EVENT_LBUTTONDOWN: # When we click left button of mouse we can draw a new park area.
        pos_list.append((x,y))
    
    if events==cv2.EVENT_RBUTTONDOWN: # When we click right button of mouse we can delete park area.
        for i,pos in enumerate(pos_list):
            x1,y1=pos
            if x1<x<x1+width and y1<y<y1+height:
                pos_list.pop(i)
    
    with open("CarParkPos","wb") as f: # We're reading position list.
        pickle.dump(pos_list,f)


while True:

    for pos in pos_list:
        cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),(255,0,255),2) # We can draw purple rectangles to describe.

    cv2.imshow("Image",img)

    cv2.setMouseCallback("Image",mouse_click)

    cv2.waitKey(1)



    k=cv2.waitKey(30) & 0xFF # When we click ESC, All windows destroy.
    if k==27:
        break
cv2.destroyAllWindows()

Detect, Show and Count Empty Park Area

In [7]:
cap=cv2.VideoCapture(r"C:\Users\umite\Desktop\vs_yapay_zeka\parking_space_counter\carPark.mp4") # Getting video.

with open("CarParkPos","rb") as f: # Getting position information.
    pos_list=pickle.load(f)

succes,img=cap.read()


width,height=107,48 # Width and height of only for one car parking area.

def check_parking_space(img_pro):

    space_counter=0 # Space counter

    for pos in pos_list:
        x,y=pos # Park area positions

        img_cropped=img_pro[y:y+height,x:x+width] # Crop every parking area.

        count=cv2.countNonZero(img_cropped) # We count nonZero pixels, so we can see is it full or empty.
        perc=round(count/2600,2)
        cvzone.putTextRect(img,str(perc),(x,y+height-3),scale=1,thickness=2,offset=0) # How many percentage of parking area is full?

        if count<900: # 900 is our threshold to make decision is it full or empty. If nonZero pixels are under 900, it's empty. Else not.
            color=(0,255,0)
            thickess=5
            space_counter=space_counter+1 # If it's empty we add to count empty parking areas.

        else:
            color=(0,0,255)
            thickess=2

        cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),color=color,thickness=thickess) # Drawing rectabgles by situation. If it's empty rectangle is green, else red.
    
    cvzone.putTextRect(img,f"Free: {space_counter}/{len(pos_list)}",(100,50),scale=3,thickness=5,offset=20,colorR=(0,220,0)) # Typing number of free parking areas.



while True:

    if cap.get(cv2.CAP_PROP_POS_FRAMES)==cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES,0)

    succes,img=cap.read()

    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # Color transformation BGR to GRAY.
    img_blurred=cv2.GaussianBlur(img_gray,(3,3),1) # Blurring image.
    img_thresholded=cv2.adaptiveThreshold(img_blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,16) # All image black or white by thresholding.
    img_median=cv2.medianBlur(img_thresholded,5) # It's clear when we make medianBlur.
    kernel = np.ones((3, 3), np.uint8)
    img_dilated=cv2.dilate(img_median,kernel,iterations=1) # Dilation for better results.

    
    check_parking_space(img_dilated)

    cv2.imshow("Video",img)
    #cv2.imshow("Video Blurred",img_blurred)
    #cv2.imshow("Video Thresholded",img_thresholded)
    #cv2.imshow("Video Median",img_median)
    #cv2.imshow("Video Dilated",img_dilated)

    k=cv2.waitKey(30) & 0xFF
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()

